# 6.2 Стекинг

Разберём стекинг на практике, проанализировав датасет, описывающий параметры, которые были сняты со спутника при фотографировании Земли. У нас есть 54 переменные. Для упрощения будем рассматривать два типа поверхностей —так мы сводим нашу задачу к задаче бинарной классификации. Сделаем базовую предобработку, воспользуемся StandardScaler.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.base import clone

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from matplotlib import pyplot as plt

%matplotlib inline

/var/folders/t1/pt8lfl0s2rz_z_7z9cbm9f5r0000gn/T/ipykernel_37429/2914329483.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz', sep=',', header=None)[:10000]

In [3]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2


In [4]:
df.shape

(10000, 55)

In [5]:
features = list(range(0, 54))
target = 54

df = df[(df[target] == 1) | (df[target] == 2)]

In [6]:
cover_train, cover_test = train_test_split(df, test_size=0.5)

cover_X_train, cover_y_train = cover_train[features], cover_train[target]
cover_X_test, cover_y_test = cover_test[features], cover_test[target]

In [7]:
scaler = StandardScaler()
cover_X_train = scaler.fit_transform(cover_X_train)
cover_X_test = scaler.transform(cover_X_test)

Stacking — еще один способ объединить несколько алгоритмов в один, который часто используется как в решении реальных задач из промышленной сферы, так и в конкурсах на платформах вроде Kaggle.  
Подход использует понятие _базовых классификаторов_, каждый из которых независимо обучается на некотором (возможно одном и том же) множестве признаков, а также _мета-классификатора_, использующего предсказания базовых классификаторов как признаки.

Для избежания переобучения будем разбивать обучающую выборку на фолды.  
Например, фолды при разбиении на три части:  
``==*``  
``=*=``  
``*==``  

Это требуется для того, чтобы получить новые признаки (ответы алгоритмов на первом уровне) на всей обучающей выборке, т.е. ответы алгоритма на тех объектах, которые не были использованы во время обучения. В примере выше мы будем использовать ответы алгоритма, полученные на объектах звездочках. _Важно_: на каждом фолде мы обучаем алгоритм заново.

Стекинг позволяет объединять ответы нескольких алгоритмов первого уровня в один большой ответ при помощи нового алгоритма обучения. Для избежания переобучения будем разбивать обучающую выбоку на фолды — разбиение обучающей выборки на несколько частей. На каждом фолде мы обучаем алгоритм заново.

Функция compute_meta_feature принимает на вход один алгоритм и возвращает новые признаки на объектах, которые не использовались во время обучения. Функция generate_meta_feature делает подобное, но принимает на вход несколько классификаторов в списке и повторяет процедуру, а затем генерирует и возвращает матрицу с изначальным количеством объектов, признаков будет столько, сколько мы передали классификаторов. 

In [8]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features using the classifier.
    
    :arg clf: scikit-learn classifier
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

In [9]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
    """
    Generates metafeatures using a list of classifiers.
    
    :arg classifiers: list of scikit-learn classifiers
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

Обучим один градиентный бустинг, в котором будем использовать 300 алгоритмов. Мы получим accuracy = 0.7899929527836504. Если мы будем агрегировать бустинг со случайным лесом и с двумя логистическими регрессиями, то получим результат лучше: 0.8118393234672304.

In [10]:
np.random.seed(42)

In [11]:
clf = GradientBoostingClassifier(n_estimators=300)
clf.fit(cover_X_train, cover_y_train)

accuracy_score(clf.predict(cover_X_test), cover_y_test)

0.7780126849894292

In [12]:
cv = KFold(n_splits=10, shuffle=True)

stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='liblinear', max_iter=5000),
    LogisticRegression(C=0.001, penalty='l2', solver='liblinear', max_iter=5000),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=300)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|██████████| 4/4 [00:18<00:00,  4.72s/it]


In [13]:
total_features_train = np.hstack([cover_X_train, stacked_features_train])
total_features_test = np.hstack([cover_X_test, stacked_features_test])

In [14]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs')
clf.fit(stacked_features_train, cover_y_train)
accuracy_score(clf.predict(stacked_features_test), cover_y_test)

0.802677942212826

# Задания! 
Все задания выполняются на основе датасета:

In [15]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint
dataset = load_digits()
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

# Задание 6.6.1
Выше мы разобрали схему генерации признаков в стекинге, когда для тестовой выборки алгоритм заново переобучался на всей тренировочной выборке. Реализуйте схему, когда вместо этого производится агрегация ответов всех обученных на фолдах классификаторов на тестовой выборке при помощи усреднения.

Логика решения:
1) Создадим X_meta_test, заполним его нулями (по аналогии с X_meta_train);
2) Далее на каждом шаге, где мы обучаем folded_clf.fit (X_fold_train, y_fold_train) и его предсказания на X_fold_predict запихиваем в X_meta_train[predict_fold_index], добавим ещё одну строку, где в X_meta_test будем добавлять предсказания вероятностей folded_clf на X_test. Их можно сразу складывать друг с другом или сохранить много массивов, тогда в конце их нужно будет все сложить, а потом делить на количество сплитов (количество массивов равно количеству сплитов в кросс-валидации);
3) После цикла останется только усреднить все эти массивы — это и будет наш X_meta_test.
За основу нужно взять следующий код:

In [16]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """    Эта функция подсчитывает признаки для мета-классификатора.     Они являются вероятностями классов при решении задачи многоклассовой классификации.    :arg clf: классификатор    :args X_train, y_train: обучающая выборка    :arg X_test: признаки тестовой выборки    :arg cv: класс, генерирующий фолды (KFold)    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)

    return X_meta_train, X_meta_test


In [17]:
# Напишите ваш код ниже
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора.
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    X_meta_test = np.zeros((len(X_test), n_classes), dtype=np.float32)
    
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        X_meta_test += folded_clf.predict_proba(X_test)
    
    X_meta_test /= cv.get_n_splits()

    return X_meta_train, X_meta_test

Для следующих заданий используйте этот код:

In [18]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [19]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

In [20]:
def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

ПОДСКАЗКА
Во всех случаях, когда модель принимает random_state — обязательно указывайте его равным 42
На разных версиях sklearn ответы могут отличаться
В задании может понадобиться, а может не понадобиться нормализация и это нужно проверить во время решения задания.

# Задание 6.6.2

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимое количество итераций — 2000
логистическая регрессия с L2-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — multinomial, максимальное допустимое количество итераций — 2000
случайный лес из 300 деревьев
градиентный бустинг из 200 деревьев
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [18]:
def compute_accuracy(clf, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round((y_test_pred == y_test).mean(), 6)

logreg_l1 = LogisticRegression(C=0.001, penalty='l1', solver='saga', max_iter=2000, multi_class='ovr', random_state=42)
logreg_l2 = LogisticRegression(C=0.001, penalty='l2', solver='saga', max_iter=2000, multi_class='multinomial', random_state=42)
rf = RandomForestClassifier(n_estimators=300, random_state=42)
gb = GradientBoostingClassifier(n_estimators=200, random_state=42)

meta_clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto', random_state=42, max_iter=10000)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features(
    classifiers=[logreg_l1, logreg_l2, rf, gb],
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    cv=kf
)

meta_clf_accuracy = compute_accuracy(meta_clf, stacked_features_train, y_train, stacked_features_test, y_test)
print("Meta-classifier accuracy:", meta_clf_accuracy)

100%|██████████| 4/4 [01:30<00:00, 22.60s/it]

Meta-classifier accuracy: 0.977778


# Задание 6.6.3

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
случайный лес из 300 деревьев
случайный лес из 200 экстремальных деревьев
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [19]:
rf_300 = RandomForestClassifier(n_estimators=300, random_state=42)
et_200 = ExtraTreesClassifier(n_estimators=200, random_state=42)

meta_clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto', random_state=42, max_iter=10000)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features(
    classifiers=[rf_300, et_200],
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    cv=kf
)

meta_clf_performance = compute_metric(meta_clf, stacked_features_train, y_train, stacked_features_test, y_test)
print("Meta-classifier performance:", meta_clf_performance)

100%|██████████| 2/2 [00:08<00:00,  4.08s/it]


Meta-classifier performance: 0.980652


# Задание 6.6.4

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
метод ближайшего соседа (k-NN) со стандартными параметрами
случайный лес из 300 экстремальных деревьев
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [22]:
knn = KNeighborsClassifier()
et_300 = ExtraTreesClassifier(n_estimators=300, random_state=42)

meta_clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto', random_state=42, max_iter=10000)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features(
    classifiers=[knn, et_300],
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    cv=kf
)

meta_clf_performance = compute_metric(meta_clf, stacked_features_train, y_train, stacked_features_test, y_test)
print("Meta-classifier performance:", meta_clf_performance)

100%|██████████| 2/2 [00:10<00:00,  5.06s/it]


Meta-classifier performance: 0.987201


# Задание 6.6.5

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимоей количество итераций — 2000
метод ближайшего соседа со стандартными параметрами
случайный лес из 300 экстремальных деревьев
AdaBoost со стандартными параметрами
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [24]:
logreg_l1 = LogisticRegression(C=0.001, penalty='l1', solver='saga', max_iter=2000, multi_class='ovr', random_state=42)
knn = KNeighborsClassifier()
et_300 = ExtraTreesClassifier(n_estimators=300, random_state=42)
ada = AdaBoostClassifier(random_state=42)

meta_clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto', random_state=42, max_iter=10000) 

kf = KFold(n_splits=5, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features(
    classifiers=[logreg_l1, knn, et_300, ada],
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    cv=kf
)

meta_clf_performance = compute_metric(meta_clf, stacked_features_train, y_train, stacked_features_test, y_test)
print("Meta-classifier performance:", meta_clf_performance)

100%|█████████████████████████████████████████████| 4/4 [00:21<00:00,  5.38s/it]

Meta-classifier performance: 0.987404



/Users/admin/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


# Задание 6.6.6

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
случайный лес из 300 деревьев
случайный лес из 300 экстремальных деревьев
Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).
Для корректной работы необходимо подправить код в функции compute_meta_feature. Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric. Количество фолдов = 10

In [23]:
from sklearn.model_selection import StratifiedKFold

def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    for train_fold_index, predict_fold_index in cv.split(X_train, y_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test


rf_300 = RandomForestClassifier(n_estimators=300, random_state=42)
et_300 = ExtraTreesClassifier(n_estimators=300, random_state=42)

meta_clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto', random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features(
    classifiers=[rf_300, et_300],
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    cv=skf
)

meta_clf_performance = compute_metric(meta_clf, stacked_features_train, y_train, stacked_features_test, y_test)
print("Meta-classifier performance:", meta_clf_performance)

100%|██████████| 2/2 [04:48<00:00, 144.25s/it]


Meta-classifier performance: 0.983918


# Задание 6.6.7

В последнем пункте предыдущей задачи измените 10 фолдов на 20. Укажите полученное качество.

In [24]:
skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features(
    classifiers=[rf_300, et_300],
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    cv=skf
)

meta_clf_performance = compute_metric(meta_clf, stacked_features_train, y_train, stacked_features_test, y_test)
print("Meta-classifier performance with 20 folds:", meta_clf_performance)

100%|██████████| 2/2 [01:00<00:00, 30.38s/it]


Meta-classifier performance with 20 folds: 0.984228


# Задание 6.6.8

В предыдущей задаче укажите количество фолдов равным 5 и поменяйте мета-алгоритм на случайный лес со стандартными параметрами. Укажите полученное качество.

In [25]:
from sklearn.ensemble import RandomForestClassifier

meta_clf = RandomForestClassifier(random_state=42)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features(
    classifiers=[rf_300, et_300],
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    cv=skf
)

meta_clf_performance = compute_metric(meta_clf, stacked_features_train, y_train, stacked_features_test, y_test)
print("Meta-classifier performance with 5 folds using Random Forest meta-algorithm:", meta_clf_performance)

100%|██████████| 2/2 [00:17<00:00,  8.85s/it]


Meta-classifier performance with 5 folds using Random Forest meta-algorithm: 0.981661


# Задание 6.6.9

В предыдущей задаче поменяйте мета-алгоритм на метод ближайших соседей (k-NN) со стандартными параметрами. Укажите полученное качество.

In [26]:
from sklearn.neighbors import KNeighborsClassifier

meta_clf = KNeighborsClassifier()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features(
    classifiers=[rf_300, et_300],
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    cv=skf
)

meta_clf_performance = compute_metric(meta_clf, stacked_features_train, y_train, stacked_features_test, y_test)
print("Meta-classifier performance with 5 folds using k-NN meta-algorithm:", meta_clf_performance)

100%|██████████| 2/2 [00:16<00:00,  8.19s/it]

Meta-classifier performance with 5 folds using k-NN meta-algorithm: 0.98417


# Задание 6.6.10

В предыдущей задаче поменяйте мета-алгоритм на градиентный бустинг со стандартными параметрами. Укажите полученное качество.

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

meta_clf = GradientBoostingClassifier(random_state=42)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features(
    classifiers=[rf_300, et_300],
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    cv=skf
)

meta_clf_performance = compute_metric(meta_clf, stacked_features_train, y_train, stacked_features_test, y_test)
print("Meta-classifier performance with 5 folds using Gradient Boosting meta-algorithm:", meta_clf_performance)

100%|██████████| 2/2 [00:15<00:00,  7.57s/it]


Meta-classifier performance with 5 folds using Gradient Boosting meta-algorithm: 0.984925


# Задание 6.6.11

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
случайный лес из 300 экстремальных деревьев
Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).
Для генерации фолдов используйте класс StratifiedKFold и поправленный Вами ранее код в функции compute_meta_feature.
Выполните разбиение на 3 фолда.
Как мета-алгортм используйте случайный лес из 100 экстремальных деревьев. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [28]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold

rf_gini_300 = RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=24, random_state=42)
et_300 = ExtraTreesClassifier(n_estimators=300, random_state=42)

meta_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features(
    classifiers=[rf_gini_300, et_300],
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    cv=skf
)

meta_clf_performance = compute_metric(meta_clf, stacked_features_train, y_train, stacked_features_test, y_test)
print("Meta-classifier performance with 3 folds using Extra Trees meta-algorithm:", meta_clf_performance)

100%|██████████| 2/2 [00:09<00:00,  4.85s/it]


Meta-classifier performance with 3 folds using Extra Trees meta-algorithm: 0.986498


# Задание 6.6.12

Обучите на тренировочной выборке следующие алгоритмы:
случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
случайный лес из 300 экстремальных деревьев
логистическую регрессию со стандартными параметрами
Усредните их ответы на тестовой выборке методом сложения предсказаний и затем взятия функции argmax: answer = (prediction1 + prediction2 + prediction3).argmax(axis = 1).
Посчитайте качество, аналогично функции compute_metric (F1-score с макро-усреднением, округленный до 6 знака).

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

rf_gini_300 = RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=24, random_state=42)
et_300 = ExtraTreesClassifier(n_estimators=300, random_state=42)
logreg = LogisticRegression()

rf_gini_300.fit(X_train, y_train)
et_300.fit(X_train, y_train)
logreg.fit(X_train, y_train)

rf_pred = rf_gini_300.predict_proba(X_test)
et_pred = et_300.predict_proba(X_test)
logreg_pred = logreg.predict_proba(X_test)

average_pred = (rf_pred + et_pred + logreg_pred) / 3

# Define compute_metric function
def compute_metric(y_pred_proba, y_true):
    y_pred = np.argmax(y_pred_proba, axis=1)
    return np.round(f1_score(y_true, y_pred, average='macro'), 6)

f1_macro = compute_metric(average_pred, y_test)
print("F1-score with macro averaging:", f1_macro)

/Users/adil/PycharmProjects/codes/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


F1-score with macro averaging: 0.976259
